In [1]:
#json simple dic-2-ay-es

import json
import os

file_aymtrada = "data/diccionarios/Aymara/dic-2-ay-es.json"  
archivo_salida = "data/compare/Aymara/dic-2-ay-es-simple.json" 

with open(file_aymtrada, 'r', encoding='utf-8') as f:
    datos = json.load(f)

resultado = []
palabras = datos['palabras']

for clave, entrada in palabras.items():
    if 'palabra' in entrada and 'definiciones' in entrada:
        palabra = entrada['palabra']
            
        for def_obj in entrada['definiciones']:
            if isinstance(def_obj, dict) and 'definicion' in def_obj:
                definicion = def_obj['definicion']
                resultado.append({
                    "Aymara": palabra,
                    "Spanish": definicion
                })

with open(archivo_salida, 'w', encoding='utf-8') as f:
    json.dump(resultado, f, ensure_ascii=False, indent=2)
        
       

In [2]:
#simple dic 1 html to json  dic-1-ay-es

import json
import re

archivo_html = "data/fuentes/Aymara/dic_1.html" 
archivo_salida = "data/compare/Aymara/dic-1-ay-es-simple.json"

with open(archivo_html, 'r', encoding='utf-8') as f:
    html_content = f.read()

filas_datos = re.findall(r'<tr>\s*<td[^>]*>([^<]+)</td>\s*<td[^>]*>([^<]+)</td>', html_content)

resultado = []

for Aymara, Spanish in filas_datos:
    resultado.append({
        "Aymara": Aymara.strip(),
        "Spanish": Spanish.strip()
    })

with open(archivo_salida, 'w', encoding='utf-8') as f:
    json.dump(resultado, f, ensure_ascii=False, indent=2)


In [3]:
# Comparar dic1 - dic2 - Versión de debug simple
import json

JSON1 = "data/compare/aymara/dic-1-ay-es-simple.json"
JSON2 = "data/compare/aymara/dic-2-ay-es-simple.json"
out = "data/diccionarios/aymara/diferences-ay-es-dic1-dic2.json" 

with open(JSON1, encoding='utf8') as f1, open(JSON2, encoding='utf8') as f2:
    datos1, datos2 = json.load(f1), json.load(f2)

def extraer_palabras_simple(datos):
    """Versión simplificada para debug"""
    palabras = set()
    objetos = []
    
    if isinstance(datos, list):
        for item in datos:
            if 'Aymara' in item and item['Aymara']:
                palabras.add(item['Aymara'].strip().lower())
                objetos.append(item)
    return palabras, objetos

palabras1, objetos1 = extraer_palabras_simple(datos1)
palabras2, objetos2 = extraer_palabras_simple(datos2)

print(f"Debug - Palabras en dic1: {len(palabras1)}")
print(f"Debug - Palabras en dic2: {len(palabras2)}")

only_1 = palabras1 - palabras2
only_2 = palabras2 - palabras1
common = palabras1 & palabras2

print(f"Debug - Solo en dic1: {len(only_1)}")
print(f"Debug - Solo en dic2: {len(only_2)}")
print(f"Debug - Comunes: {len(common)}")

objetos_only_1 = [obj for obj in objetos1 if obj['Aymara'].strip().lower() in only_1]
objetos_only_2 = [obj for obj in objetos2 if obj['Aymara'].strip().lower() in only_2]
objetos_common = [obj for obj in objetos1 + objetos2 if obj['Aymara'].strip().lower() in common]

results_json = {
    "results": {
        "total_dic1": len(palabras1),
        "total_dic2": len(palabras2),
        "total_only_dic1": len(only_1),
        "total_only_dic2": len(only_2),
        "total_common": len(common)
    },
    "only_dic1": {
        "palabras": sorted(list(only_1)),
    },
    "only_dic2": {
        #"palabras": sorted(list(only_2)),
    }#,
    #"common": {
    #    "palabras": sorted(list(common)),
     
    #}
}

with open(out, 'w', encoding='utf-8') as f:
    json.dump(results_json, f, indent=2, ensure_ascii=False)

print(f"\nResultados finales:")
print(f"Total only Dic1: {len(only_1)}")
print(f"Total only Dic2: {len(only_2)}")
print(f"Both Dic1 - Dic2: {len(common)}")

Debug - Palabras en dic1: 4970
Debug - Palabras en dic2: 6512
Debug - Solo en dic1: 3575
Debug - Solo en dic2: 5117
Debug - Comunes: 1395

Resultados finales:
Total only Dic1: 3575
Total only Dic2: 5117
Both Dic1 - Dic2: 1395


In [ ]:
import json
import re

json_dic = "data/compare/aymara/dic-1and2.json"
file_aym = "americasnlp2025/ST1_MachineTranslation/data/aymara-spanish/train.aym" 
out = "data/compare/aymara/dic1and2_vs_archivo.json"


with open(json_dic, encoding='utf8') as f:
    datos_dic = json.load(f)

with open(file_aym, encoding='utf8') as f:
    contenido_en = f.read()

palabras_dic = {} 
objetos_dic = []

if isinstance(datos_dic, list):
    for item in datos_dic:
        if 'Aymara' in item and item['Aymara']:
            palabra_original = item['Aymara'].strip()
            palabra_lower = palabra_original.lower()
            if palabra_lower not in palabras_dic:
                palabras_dic[palabra_lower] = palabra_original
            objetos_dic.append(item)

encontradas = []
no_encontradas = []
debug_info = {}

for palabra_lower, palabra_original in palabras_dic.items():
    patron = rf'\b{re.escape(palabra_lower)}\b'
    match = re.search(patron, contenido_en, re.IGNORECASE)
    
    if match:
        encontradas.append(palabra_original)
    else:
        no_encontradas.append(palabra_original)
        simple_search = palabra_lower in contenido_en.lower()


objetos_encontrados = [obj for obj in objetos_dic if obj['Aymara'].strip().lower() in [p.lower() for p in encontradas]]
objetos_no_encontrados = [obj for obj in objetos_dic if obj['Aymara'].strip().lower() in [p.lower() for p in no_encontradas]]

resultado = {
    "no_encontradas": sorted(no_encontradas),
}

with open(out, 'w', encoding='utf-8') as f:
    json.dump(resultado, f, indent=2, ensure_ascii=False)

print(f"Total: {len(palabras_dic)}")
print(f"Find: {len(encontradas)}")
print(f"No exits: {len(no_encontradas)}")


Total: 10087
Encontradas: 623
No encontradas: 9464
